# Important Instructions:
- YOLOv8 only works on __Google Colab__ without any extra changes. But it will work on you local computer if you use Visual Studio Code and run the notebook in a virtual environment.
- The only input the notbook expects is the __path to the folder you have extracted the folder in__.

In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 646.6/646.6 kB 6.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)
print('Connecting to Drive....')

%cd /content/gdrive/Othercomputers/My Computer/aryan/Pedestrian Crossing Detection
print('Done!')

Mounted at /content/gdrive
Connecting to Drive....
/content/gdrive/Othercomputers/My Computer/aryan/Pedestrian Crossing Detection
Done!


In [3]:
from ultralytics import YOLO

In [4]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.221 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.9/78.2 GB disk)


# Detection

In [5]:
import glob
import os
import shutil
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

#Bounding Box Merge Algorithm
def merge_bounding_boxes(bounding_boxes,image_width,image_height):
  [x,y] = [image_height,image_width]
  [w,h] = [0,0]
  for i in range(0,bounding_boxes.shape[0]):
    if(bounding_boxes.ndim == 1):
      center_x_1 = int(bounding_boxes[0]*image_width)
      center_y_1 = int(bounding_boxes[1]*image_height)
      width_1 = int(bounding_boxes[2]*image_width)
      height_1 = int(bounding_boxes[3]*image_height)
      x = int(center_x_1 - (width_1/2))
      y = int(center_y_1 - (height_1/2))
      w = int(center_x_1 + (width_1/2))
      h = int(center_y_1 + (height_1/2))
    else:
      center_x_1 = int(bounding_boxes[i,0]*image_width)
      center_y_1 = int(bounding_boxes[i,1]*image_height)
      width_1 = int(bounding_boxes[i,2]*image_width)
      height_1 = int(bounding_boxes[i,3]*image_height)
      vert_x = int(center_x_1 - (width_1/2))
      vert_y = int(center_y_1 - (height_1/2))
      vert_w = int(center_x_1 + (width_1/2))
      vert_h = int(center_y_1 + (height_1/2))
      if(vert_x < x):
        x = vert_x
      if(vert_y < y):
        y = vert_y
      if(vert_w > w):
        w = vert_w
      if(vert_h > h):
        h = vert_h
    return np.array([x,y,w,h])

In [6]:
!yolo task=detect mode=predict model= "Object_detection_best.pt" source= 'detection/dataset' imgsz=640 save=False save_txt=True save_conf=True

Ultralytics YOLOv8.0.221 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126745 parameters, 0 gradients, 28.4 GFLOPs

image 1/790 /content/gdrive/Othercomputers/My Computer/aryan/Pedestrian Crossing Detection/detection/dataset/detection_set04_video_0005_04260.png: 640x640 1 not-crossing, 16.3ms
image 2/790 /content/gdrive/Othercomputers/My Computer/aryan/Pedestrian Crossing Detection/detection/dataset/detection_set04_video_0005_04290.png: 640x640 (no detections), 16.3ms
image 3/790 /content/gdrive/Othercomputers/My Computer/aryan/Pedestrian Crossing Detection/detection/dataset/detection_set04_video_0005_04320.png: 640x640 (no detections), 16.3ms
image 4/790 /content/gdrive/Othercomputers/My Computer/aryan/Pedestrian Crossing Detection/detection/dataset/detection_set04_video_0005_04350.png: 640x640 2 not-crossings, 16.3ms
image 5/790 /content/gdrive/Othercomputers/My Computer/aryan/Pedestrian Crossing Detection/detection/dataset/detect

In [ ]:
import time

# Delay for 2 mins so the google drive waits for files to load
time.sleep(120)

In [7]:
# Create the 'training_dataset' directory if it doesn't exist
training_dataset_directory = os.path.join('runs', 'detect', 'predict', 'images')
os.makedirs(training_dataset_directory) if not os.path.exists(training_dataset_directory) else None

In [8]:
path = 'runs/detect/predict/labels/'
img_path = 'detection/dataset'
save_path = 'runs/detect/predict/images'

for fname in os.listdir(save_path):
  if fname.endswith('.png'):
    print("The path already contains images, skipping this part.")
    break
else:
  from distutils.dir_util import copy_tree
  copy_tree(img_path, save_path)
  print("Path Copied!")

Path Copied!


In [9]:
import time

# Delay for 2 mins so the google drive waits for files to load
time.sleep(120)

In [10]:
class_dict = {"crossing": 0, "not-crossing": 1, "crossing-irrelevant": 2}

# class_colors = {
#     0: (0, 0, 255),   # Red for "crossing"
#     1: (255, 0, 0),   # Blue for "not-crossing"
#     2: (0, 255, 0)    # Green for "crossing-irrelevant"
# }

class_colors = {
    0: (0, 0, 255),   # Red for "crossing"
    1: (0, 255, 0),   # Green for "not-crossing"
    2: (255, 0, 0)    # Blue for "crossing-irrelevant"
}

# Make sure the save directory exists
os.makedirs(save_path, exist_ok=True)

# Process each text file
for filename in glob.glob(os.path.join(path, '*.txt')):
    with open(os.path.join(os.getcwd(), filename), 'r') as f:
        detected_objects = []  # List to store information about each detected object
        image_name = os.path.basename(filename).replace('txt', 'png')

        for line in f:
            cl, label_x, label_y, label_w, label_h, conf = line.split(' ')
            cl = int(cl)
            b = float(conf)
            a = np.array([float(label_x), float(label_y), float(label_w), float(label_h)])

            detected_objects.append({
                'class_index': cl,
                'confidence': b,
                'bounding_box': a
            })

        # Image Elaboration
        image = cv.imread(os.path.join(img_path, image_name))
        [image_height, image_width, levels] = image.shape

        # Process each detected object
        for obj in detected_objects:
            detected_class_index = obj['class_index']
            detected_class_label = list(class_dict.keys())[list(class_dict.values()).index(detected_class_index)]

            # Retrieve bounding box, confidence, and color
            bounding_box = obj['bounding_box']
            conf = obj['confidence']
            color = class_colors.get(detected_class_index, (255, 255, 255))  # Default to white if color not found

            [x, y, w, h] = merge_bounding_boxes(bounding_box, image_width, image_height)

            # Create Rect
            cv.rectangle(image, (x, y), (w, h), color, 1)
            cv.putText(image, f'{detected_class_label} {conf:.2f}', (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)

        # Saving Image
        print('Saving ' + image_name)
        cv.imwrite(os.path.join(save_path, image_name), image)

Saving detection_set04_video_0005_04260.png
Saving detection_set04_video_0005_04350.png
Saving detection_set04_video_0005_04440.png
Saving detection_set04_video_0005_05250.png
Saving detection_set04_video_0005_05280.png
Saving detection_set04_video_0005_05340.png
Saving detection_set04_video_0005_05370.png
Saving detection_set04_video_0005_05400.png
Saving detection_set04_video_0005_05430.png
Saving detection_set04_video_0005_05460.png
Saving detection_set04_video_0005_05490.png
Saving detection_set04_video_0005_05520.png
Saving detection_set04_video_0005_05550.png
Saving detection_set04_video_0005_05580.png
Saving detection_set04_video_0005_05610.png
Saving detection_set04_video_0005_05640.png
Saving detection_set04_video_0005_05670.png
Saving detection_set04_video_0005_05700.png
Saving detection_set04_video_0005_05730.png
Saving detection_set04_video_0005_05760.png
Saving detection_set04_video_0005_05790.png
Saving detection_set04_video_0005_05820.png
Saving detection_set04_video_000